In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

The goal is to create an effective Collaborative Filtering Model from scratch. We will use PyTorch as automatic differentiation and gpu programming tool.

## Preprocessing

We will use MovieLens dataset http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

It contains 100,000 ratings applied to 9,000 movies by 700 users.

In [2]:
PATH = os.getcwd() + '/data'

In [3]:
df = pd.read_csv(PATH + '/ratings.csv')

In [4]:
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


userId and movieId are categorial variables that we will use as predictors and rating is the dependant variable, the outcome that we want to predict.

In [5]:
df.shape

(100004, 4)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


There are not missing values

In [7]:
df.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


Rating values range is (0.5 - 5).

movies ids are not contiguous, they start at 1 and go to 163949. This is not efficient when creating an embedding matrix. So, we get a list of unique movieId and then we create a mapping from every movieId to a contiguous integer. We also apply the same steps to userId just to be cautious.

In [8]:
users = df['userId'].unique()
movies = df['movieId'].unique()

user2idx = {k: i for i, k in enumerate(users)}
movie2idx = {k: i for i, k in enumerate(movies)}

df['userId'] = df['userId'].apply(lambda x: user2idx[x])
df['movieId'] = df['movieId'].apply(lambda x: movie2idx[x])

Number of users and movies.

In [9]:
n_users = len(df['userId'].unique())
n_movies = len(df['movieId'].unique())

In [10]:
n_users, n_movies

(671, 9066)

We will not use timestamp column.

In [11]:
df.drop('timestamp', axis=1, inplace=True)

## Train, valid split

We split the data into a standard 80 % training and 20 % validation.

In [12]:
n = df.shape[0]
np.random.seed = 42
n_val = int(n * 0.2)
idxs = np.random.permutation(n)
# validation indexes
val_idxs = idxs[:n_val]

In [13]:
mask = np.zeros(len(df), dtype=bool)
mask[np.array(val_idxs)] = True

In [14]:
torch.cuda.is_available()

True

We create training and validation PyTorch dataloaders. They are a useful structure to get mini batches of data when fitting the model.

In [15]:
batch_size = 64
train_dl = DataLoader(df[~mask].values, batch_size, shuffle=True, num_workers=1)
valid_dl = DataLoader(df[mask].values, batch_size * 2, shuffle=False, num_workers=1)

We are going to represent userId and movieId in a high dimentional space where each of them will be represented by a vector of n dimensions (number of factors). Then we will predict the rating of a particular user on a particular movie by calculating the dot product between their vectors.

When we are deciding if user X would like movie Z, we are saying:
<br/>Which are the users that enjoyed movies like Z ? 
<br/>And which are the movies that were liked by people like user X?

## Embeddings dot product

Given a userId and a movieId, we want to look up into our users and embedding matrices to find their vectors of n factors and then take the dot product.

We are going to create a PyTorch Module, something that we can use as a layer and as neural net.
<br/>To define a module we need to create a Python class. PyTorch does not reinvent totally new ways of doing things as for example TensorFlow does. In PyTorch we tend to use pythonic ways to do things. 

In [16]:
class EmbeddingDotProd(nn.Module):
    def __init__(self, n_users, n_movies, n_factors):
        super().__init__()
        # Users embedding layer: dim = number of users x number of factors. 
        self.users = nn.Embedding(n_users, n_factors)
        # Movies embedding layer: dim = number of movies x number of factors. 
        self.movies = nn.Embedding(n_movies, n_factors)
        
        # The two embedding matrices are randomly initialized (we do not use any pre-trained vectors)
        # But it is important to randomly initialize them to a reasonable set of numbers / scale.
        self.users.weight.data.uniform_(0, 0.05)
        self.movies.weight.data.uniform_(0, 0.05)
    
    def forward(self, data):
        u, m = data[:, 0], data[:, 1]
        # Compute dot product between users vectors and movies vectors.
        return (self.users(u) * self.movies(m)).sum(1)

In [17]:
def fit(model, train_dl, n_epochs, optim, criterion):
    bar = tqdm_notebook(total=n_epochs)
    
    avg_mom = 0.98
    avg_loss = 0.
    batch_num = 0
    
    for epoch in range(n_epochs):
        for i, batch in enumerate(train_dl):
            batch_num += 1
            
            inp = Variable(batch[:, :2].long()).cuda()
            targ = Variable(batch[:, 2].float()).cuda()
        
            # Forward pass: compute predicted y by passing x to the model.
            pred = model(inp)

            # Compute loss: we pass Variables containing the predicted and true values of y.
            loss = criterion(pred, targ)

            # Zero the gradients before running the backward pass.
            opt.zero_grad()

            # Backward pass: compute gradient of the loss with respect to all the learnable parameters of the model.
            loss.backward()

            # Calling the step function on an Optimizer makes an update to its parameters.
            optim.step()
            
            # Exponentially weighted moving average, to make the reported loss more stable.
            avg_loss = avg_loss * avg_mom + loss.data[0] * (1-avg_mom)
            
            # Compute bias-corrected loss estimate.
            debias_loss = avg_loss / (1 - avg_mom**batch_num)
        
        # Compute validation loss.
        val = validate(model, valid_dl, criterion)
        
        print(np.round([epoch, debias_loss] + val, 6))    
        bar.update()

In [18]:
def validate(model, dl, criterion):
    loss = []

    for i, batch in enumerate(dl):
        inp = Variable(batch[:, :2].long()).cuda()
        targ = Variable(batch[:, 2].float()).cuda()
        
        # In order to compute validation loss we only do forward pass.
        pred = model(inp)
        
        loss.append(criterion(pred, targ).data[0])
    
    return [np.mean(loss)]

In [19]:
model = EmbeddingDotProd(n_users, n_movies, 50).cuda()
opt = optim.Adam(model.parameters(), 1e-3)

In [20]:
fit(model, train_dl, 5, opt, nn.MSELoss())

[0.       2.538217 2.357314]
[1.       1.298426 1.410683]
[2.       1.018669 1.208186]
[3.       0.871076 1.145711]
[4.       0.807154 1.116886]



Loss used during training is Mean Squared Error. Let's calculate Root Mean Squared Error.

In [23]:
math.sqrt(1.116886)

1.056828273656605

## Adding bias

For instance, a user could be pretty enthusiastic and have higher ratings on average, or some movie could be popular. We want to have a constant for users and a constant for movies that shape those features (in neural networks we call that a bias).

In [25]:
min_rating = df['rating'].min()
max_rating = df['rating'].max()

In [26]:
def get_emb(num_embeddings, embedding_dim):
    emb = nn.Embedding(num_embeddings, embedding_dim)
    emb.weight.data.uniform_(-0.01, 0.01)
    return emb

In [27]:
class EmbeddingDotProdBias(nn.Module):
    def __init__(self, n_users, n_movies, n_factors):
        super().__init__()
        self.users = get_emb(n_users, n_factors)
        self.movies = get_emb(n_movies, n_factors)
        # Bias embedding layer for users: dim = number of users x 1.
        self.users_b = get_emb(n_users, 1)
        # Bias embedding layer for movies: dim = number of movies x 1.
        self.movies_b = get_emb(n_movies, 1)
    
    def forward(self, data):
        u, m = data[:, 0], data[:, 1]
        # Compute dot product between users vectors and movies vectors.
        um = (self.users(u) * self.movies(m)).sum(1)
        # Add bias
        res = um + self.users_b(u).squeeze() + self.movies_b(m).squeeze()
        # Transform results to desired output scale: min_rating - max_rating.
        res = F.sigmoid(res) * (max_rating - min_rating) + min_rating
        return res

In [28]:
# L2 regularization.
weight_decay = 1e-4
model = EmbeddingDotProdBias(n_users, n_movies, 50).cuda()
opt = optim.Adam(model.parameters(), 1e-3, weight_decay=weight_decay)

In [29]:
fit(model, train_dl, 5, opt, nn.MSELoss())

[0.       0.930192 0.896728]
[1.       0.766666 0.816125]
[2.       0.776622 0.7911  ]
[3.       0.661314 0.778883]
[4.       0.634662 0.772037]


In [30]:
math.sqrt(0.772037)

0.8786563605870045

This approach does not use a matrix factorization (i.e. creating a matrix with a row for each user and a column for each movie). In that case, any empty user-movie combination is filled with zero. As a result we get a sparse matrix.

<br/>The probabilistic matrix factorization approach that is used here takes advantage of the fact that our data structure actually looks like a typical table rather than a cross tab, therefore is only calculated the loss for user - movie combinations that actually appears.

## Save weights

We save learned weights for future visualizations

In [31]:
model.parameters

<bound method Module.parameters of EmbeddingDotProdBias(
  (users): Embedding(671, 50)
  (movies): Embedding(9066, 50)
  (users_b): Embedding(671, 1)
  (movies_b): Embedding(9066, 1)
)>

In [42]:
pickle.dump(model.users, open(f'data/users_emb.pickle', 'wb')) 
pickle.dump(model.movies, open(f'data/movies_emb.pickle', 'wb'))
pickle.dump(model.users_b, open(f'data/users_bias.pickle', 'wb')) 
pickle.dump(model.movies_b, open(f'data/movies_bias.pickle', 'wb')) 